In [1]:
import os

def find_files(directory):
    fits = []
    assn = []
    for root, dirs, files in os.walk(directory):
        for f in files:
            if f.endswith('.fits'):
                fits.append(f)
            elif f.endswith('.json'):
                assn.append(f)
          
    found_files = {'fits': sorted(fits), 'assn': sorted(assn)}
    return found_files

In [2]:
PATH = "/grp/jwst/ssb/test_build7.1/examples_for_dms"
FILES = find_files(PATH)
print(FILES)
FITS = FILES['fits']
ASSN = FILES['assn']

{'fits': ['cal_001_cal.fits', 'cal_001_cal.fits', 'cal_001_cal_old.fits', 'cal_001_rate.fits', 'cal_002_cal.fits', 'cal_002_cal.fits', 'cal_002_cal_old.fits', 'cal_002_rate.fits', 'cal_003_cal.fits', 'cal_003_cal.fits', 'cal_003_cal_old.fits', 'cal_003_rate.fits', 'cal_004_cal.fits', 'cal_004_cal.fits', 'cal_004_cal_old.fits', 'cal_004_rate.fits', 'cal_005_cal.fits', 'cal_006_cal.fits', 'cal_007_cal.fits', 'cal_008_cal.fits', 'cal_009_cal.fits', 'cal_010_cal.fits', 'cal_011_cal.fits', 'cal_012_cal.fits', 'cal_013_cal.fits', 'cal_014_cal.fits', 'cal_015_cal.fits', 'cal_016_cal.fits', 'cal_017_cal.fits', 'det_image_ch1-short_s3d.fits', 'det_image_ch1-short_x1d.fits', 'det_image_ch2-short_s3d.fits', 'det_image_ch2-short_x1d.fits', 'dispersed_ramp_from_test.fits', 'jw00035001001_01101_00001_mirimage_cal.fits', 'jw00035001001_01101_00001_mirimage_cal.fits', 'jw00035001001_01101_00001_mirimage_rate.fits', 'jw00035001001_01101_00001_mirimage_rate.fits', 'jw00035001001_01101_00001_mirimage_x1d

In [3]:
class JWSTFile():
    
    def __init__(self, filename):
        self.filename = filename
        self.full = filename.split(".")[0]
        
        in_chunks = self.full.split("_")
        print(in_chunks)
        if len(in_chunks) < 5:
            print("{0} is not formatted properly".format(filename))
            self.formatted = False
            return
        else:
            self.formatted = True
            
        first = in_chunks[0].split("-")
        if len(first) == 1:
            first = first[0]
            self.program_id = first[2:7]
            self.obs_number = first[7:10]
            self.visit_number = first[10:13]
            self.assn_number = None
        else:
            self.program_id = first[0][2:]
            self.assn_number = first[1]
            self.obs_number = None
            self.visit_number = None
            
        second = in_chunks[1]
        if second.startswith('t'):
            self.target_id = second
            self.source_id = None
            self.visit_group = None
            self.parallel_seq = None
            self.activity_number = None
        elif second.startswith('s'):
            self.source_id = second
            self.target_id = None
            self.visit_group = None
            self.parallel_seq = None
            self.activity_number = None
        else:
            self.visit_group = second[0:2]
            self.parallel_seq = second[2]
            self.activity_number = second[3:]
            self.target_id = None
            self.source_id = None
            
        third = in_chunks[2]
        if third.isdigit():
            self.exposure_number = third
            self.instrument = None
        else:
            self.instrument = third
            self.exposure_number = None
            
        fourth = in_chunks[3:-1]
        if self.assn_number:
            as_string = str(fourth)
            self.optical_elements = as_string[1:-1]    #Strip '[]' from string
            self.detector = "N/A"
        else:
            self.detector = fourth[0]
            self.optical_elements = None
        
        det = self.detector.lower()
        if det.startswith('mir'):
            self.instrument = 'miri'
        elif det.startswith('nrc'):
            self.instrument = 'nircam'
        elif det.startswith('nis'):
            self.instrument = 'niriss'
        elif det.startswith('nrs'):
            self.instrument = 'nirspec'
        else:
            self.detector = None
            
        self.suffix = in_chunks[-1]
        
        self.tuple_ = (self.filename,
                       self.assn_number,
                       self.program_id,
                       self.instrument,
                       self.detector,
                       self.optical_elements,
                       self.target_id,
                       self.source_id,
                       self.obs_number,
                       self.visit_number,
                       self.visit_group,
                       self.parallel_seq,
                       self.activity_number,
                       self.exposure_number,
                       self.suffix
                      )
        
    def __str__(self):
        return self.filename
    
    def show_info(self):
        print("***Info for {0}***".format(self.full))
        print("Program ID:  {0}".format(self.program_id))
        print("Observation number:  {0}".format(self.obs_number))
        print("Visit number:  {0}".format(self.visit_number))
        print("Visit Group:  {0}".format(self.visit_group))
        print("Parallel Sequence ID:  {0}".format(self.parallel_seq))
        print("Activity number:  {0}".format(self.activity_number))
        print("Exposure number:  {0}".format(self.exposure_number))
        print("Detector:  {0}".format(self.detector))
        print("Suffix:  {0}".format(self.suffix))

In [4]:
import sqlite3

def connect_to_sqlite(db_file):
    conn = sqlite3.connect(':memory:')
    ref = sqlite3.connect(db_file)
    c = conn.cursor()
    
    query = "".join(line for line in ref.iterdump())

    conn.executescript(query)
    
    return c, conn

In [5]:
c, conn = connect_to_sqlite("jwstfiles.db")
for row in c.execute('SELECT * FROM ami3'):
    print(row)

('ami', 'Output of ami_analyze step', 'none')
('amiavg', 'Table of averaged fringe parameters for PSF reference or science target exposures', 'none')
('aminorm', 'Final single product for the target corrected by the calibrator', 'none')


In [6]:
import pandas as pd

def add_columns(jw_properties, db_result, stage):
    jw_properties.extend(db_result[1:])
    jw_properties.append(stage)
    return jw_properties

def add_filenames_to_db(files, c, conn):
    columns = ['file',
               'assn_number',
               'program_id',
               'instrument',
               'detector',
               'optical_elements',
               'target_id',
               'source_id',
               'obs_number',
               'visit_number',
               'visit_group',
               'parallel_seq',
               'activity_number',
               'exposure_number',
               'suffix',
               'description',
               'units',
               'stage'
              ]
    df = pd.DataFrame()
    for f in files:
        jw = JWSTFile(f)
        if not jw.formatted:
            continue
            
        jw_list = list(jw.tuple_)
        suffix = []
        suffix.extend(list(c.execute('SELECT * FROM detector1 WHERE suffix=?', [jw.suffix])))
        suffix.extend(list(c.execute('SELECT * FROM image2 WHERE suffix=?', [jw.suffix])))
        suffix.extend(list(c.execute('SELECT * FROM spec2 WHERE suffix=?', [jw.suffix])))
        suffix.extend(list(c.execute('SELECT * FROM ami3 WHERE suffix=?', [jw.suffix])))
        suffix.extend(list(c.execute('SELECT * FROM coron3 WHERE suffix=?', [jw.suffix])))
        suffix.extend(list(c.execute('SELECT * FROM image3 WHERE suffix=?', [jw.suffix])))
        suffix.extend(list(c.execute('SELECT * FROM spec3 WHERE suffix=?', [jw.suffix])))
        suffix.extend(list(c.execute('SELECT * FROM tso3 WHERE suffix=?', [jw.suffix])))
        
        if len(suffix) > 0:
            suffix = suffix[-1]
        else:
            print("Invalid file suffix provided")
            continue

        jw_list = add_columns(jw_list, suffix, '1')
        print(jw_list)
                
        new_row = pd.DataFrame(columns=columns, data=[jw_list])
        df = df.append(new_row, ignore_index=True)
                
    df.to_sql("products", conn, if_exists="replace", dtype='string')
    return df

In [7]:
frame = add_filenames_to_db(FITS, c, conn)

['cal', '001', 'cal']
cal_001_cal.fits is not formatted properly
['cal', '001', 'cal']
cal_001_cal.fits is not formatted properly
['cal', '001', 'cal', 'old']
cal_001_cal_old.fits is not formatted properly
['cal', '001', 'rate']
cal_001_rate.fits is not formatted properly
['cal', '002', 'cal']
cal_002_cal.fits is not formatted properly
['cal', '002', 'cal']
cal_002_cal.fits is not formatted properly
['cal', '002', 'cal', 'old']
cal_002_cal_old.fits is not formatted properly
['cal', '002', 'rate']
cal_002_rate.fits is not formatted properly
['cal', '003', 'cal']
cal_003_cal.fits is not formatted properly
['cal', '003', 'cal']
cal_003_cal.fits is not formatted properly
['cal', '003', 'cal', 'old']
cal_003_cal_old.fits is not formatted properly
['cal', '003', 'rate']
cal_003_rate.fits is not formatted properly
['cal', '004', 'cal']
cal_004_cal.fits is not formatted properly
['cal', '004', 'cal']
cal_004_cal.fits is not formatted properly
['cal', '004', 'cal', 'old']
cal_004_cal_old.fits i

['jw82600051001_02102_00001_nrcalong_rateints.fits', None, '82600', 'nircam', 'nrcalong', None, None, None, '051', '001', '02', '1', '02', '00001', 'rateints', '3D slope images for each integration, result of ramp fitting', 'DN/s', '1']
['jw82600051001', '02102', '00001', 'nrcalong', 'uncal']
['jw82600051001_02102_00001_nrcalong_uncal.fits', None, '82600', 'nircam', 'nrcalong', None, None, None, '051', '001', '02', '1', '02', '00001', 'uncal', 'Raw 4D exposure data', 'DN', '1']
['jw84600007001', '02101', '00001', 'nrs1', 'cal']
['jw84600007001_02101_00001_nrs1_cal.fits', None, '84600', 'nirspec', 'nrs1', None, None, None, '007', '001', '02', '1', '01', '00001', 'cal', '2D fully calibrated data averaged over all integrations', 'DN/s or mJy/arcsec2', '1']
['jw84600007001', '02101', '00001', 'nrs1', 'rate']
['jw84600007001_02101_00001_nrs1_rate.fits', None, '84600', 'nirspec', 'nrs1', None, None, None, '007', '001', '02', '1', '01', '00001', 'rate', '2D slope images derived after averagin

['jw94015004002_02109_00001_mirimage_cal.fits', None, '94015', 'miri', 'mirimage', None, None, None, '004', '002', '02', '1', '09', '00001', 'cal', '2D fully calibrated data averaged over all integrations', 'DN/s or mJy/arcsec2', '1']
['jw94015004002', '0210A', '00001', 'mirimage', 'cal']
['jw94015004002_0210A_00001_mirimage_cal.fits', None, '94015', 'miri', 'mirimage', None, None, None, '004', '002', '02', '1', '0A', '00001', 'cal', '2D fully calibrated data averaged over all integrations', 'DN/s or mJy/arcsec2', '1']
['jw94015004002', '0210B', '00001', 'mirimage', 'cal']
['jw94015004002_0210B_00001_mirimage_cal.fits', None, '94015', 'miri', 'mirimage', None, None, None, '004', '002', '02', '1', '0B', '00001', 'cal', '2D fully calibrated data averaged over all integrations', 'DN/s or mJy/arcsec2', '1']
['jw94015004002', '0210C', '00001', 'mirimage', 'cal']
['jw94015004002_0210C_00001_mirimage_cal.fits', None, '94015', 'miri', 'mirimage', None, None, None, '004', '002', '02', '1', '0C'

['jw9999947001', '02102', '00002', 'nrcb3', 'a3001', 'crfints']
['jw9999947001_02102_00002_nrcb3_a3001_crfints.fits', None, '99999', 'nircam', 'nrcb3', None, None, None, '470', '01', '02', '1', '02', '00002', 'crfints', '3D calibrated data with DQ array updated by outlier_detection step', 'DN/s', '1']
['jw9999947001', '02102', '00002', 'nrcb3', 'a3001', 'psfalign']
['jw9999947001_02102_00002_nrcb3_a3001_psfalign.fits', None, '99999', 'nircam', 'nrcb3', None, None, None, '470', '01', '02', '1', '02', '00002', 'psfalign', '4D array of PSF images aligned to individual integrations of science target images', 'none', '1']
['jw9999947001', '02102', '00002', 'nrcb3', 'a3001', 'psfsub']
['jw9999947001_02102_00002_nrcb3_a3001_psfsub.fits', None, '99999', 'nircam', 'nrcb3', None, None, None, '470', '01', '02', '1', '02', '00002', 'psfsub', 'PSF subtracted', 'none', '1']
['jw9999947001', '02102', '00002', 'nrcb3', 'calints-a3001']
Invalid file suffix provided
['jw9999947001', '02102', '00002', 'n

In [8]:
frame[40:50]

,file,assn_number,program_id,instrument,detector,optical_elements,target_id,source_id,obs_number,visit_number,visit_group,parallel_seq,activity_number,exposure_number,suffix,description,units,stage
40,jw10002001001_02101_00001_nrs1_s2d.fits,None,10002,nirspec,nrs1,None,None,None,001,001,02,1,01,00001,s2d,2D resampled+combined spectroscopic image,none,1
41,jw10002001001_02101_00001_nrs1_x1d.fits,None,10002,nirspec,nrs1,None,None,None,001,001,02,1,01,00001,x1d,Extracted 1D spectra from combined exposures,flux,1
42,jw11001001001_01101_00001_NRCA1_uncal.fits,None,11001,nircam,NRCA1,None,None,None,001,001,01,1,01,00001,uncal,Raw 4D exposure data,DN,1
43,jw11111001001_01104_00001_nrcalong_cal.fits,None,11111,nircam,nrcalong,None,None,None,001,001,01,1,04,00001,cal,2D fully calibrated data averaged over all int...,DN/s or mJy/arcsec2,1
44,jw11111001001_01104_00001_nrcalong_cal.fits,None,11111,nircam,nrcalong,None,None,None,001,001,01,1,04,00001,cal,2D fully calibrated data averaged over all int...,DN/s or mJy/arcsec2,1
45,jw11111001001_01104_00001_nrcalong_rate.fits,None,11111,nircam,nrcalong,None,None,None,001,001,01,1,04,00001,rate,2D slope images derived after averaging of all...,DN/s,1
46,jw11111001001_01104_00001_nrcalong_rate.fits,None,11111,nircam,nrcalong,None,None,None,001,001,01,1,04,00001,rate,2D slope images derived after averaging of all...,DN/s,1
47,jw11111001001_01104_00001_nrcalong_x1d.fits,None,11111,nircam,nrcalong,None,None,None,001,001,01,1,04,00001,x1d,Extracted 1D spectra from combined exposures,flux,1
48,jw11111001001_01104_00001_nrcalong_x1d.fits,None,11111,nircam,nrcalong,None,None,None,001,001,01,1,04,00001,x1d,Extracted 1D spectra from combined exposures,flux,1
49,jw80600-a3002_t001_miri_p750l_s2d.fits,a3002,80600,miri,None,'p750l',t001,None,None,None,None,None,None,None,s2d,2D resampled+combined spectroscopic image,none,1


In [43]:
for row in c.execute('SELECT * FROM products'):
    print(row)

(0, 'jw00035001001_01101_00001_mirimage_cal.fits', None, 35, 'miri', 'mirimage', None, None, None, 1, 1, 1, 1, 1, 1, 'cal', '2D fully calibrated data averaged over all integrations', 'DN/s or mJy/arcsec2', 1)
(1, 'jw00035001001_01101_00001_mirimage_cal.fits', None, 35, 'miri', 'mirimage', None, None, None, 1, 1, 1, 1, 1, 1, 'cal', '2D fully calibrated data averaged over all integrations', 'DN/s or mJy/arcsec2', 1)
(2, 'jw00035001001_01101_00001_mirimage_rate.fits', None, 35, 'miri', 'mirimage', None, None, None, 1, 1, 1, 1, 1, 1, 'rate', '2D slope images derived after averaging of all countrate integrations within the exposure', 'DN/s', 1)
(3, 'jw00035001001_01101_00001_mirimage_rate.fits', None, 35, 'miri', 'mirimage', None, None, None, 1, 1, 1, 1, 1, 1, 'rate', '2D slope images derived after averaging of all countrate integrations within the exposure', 'DN/s', 1)
(4, 'jw00035001001_01101_00001_mirimage_x1d.fits', None, 35, 'miri', 'mirimage', None, None, None, 1, 1, 1, 1, 1, 1, 'x1d'

In [31]:
import time

class JWSTAssociation():
    
    def __init__(self, filename):
        self.filename = filename
        self.full = filename.split('.')[0]
        self.formatted = True
        in_chunks = self.full.split('_')
        
        print(in_chunks)
        if len(in_chunks) < 5:
            self.formatted = False
            return
            
        first = in_chunks[0][2:].split('-')
        if len(first) == 1:
            self.formatted = False
            return
        self.program_id = first[0]
        self.ac_id = first[1]
        
        second = in_chunks[1]
        #t = time.strptime(second, '%Y%m%dT%H%M%S')
        #self.datetime = time.strftime("%Y %b %d, %X", t)
        second = second.lower().split('t')[0]
        t = time.strptime(second, '%Y%m%d')
        self.datetime = time.strftime("%Y %b %d", t)
        
        self.pipeline_element = in_chunks[2]
        
        self.number = in_chunks[3]
        
        self.tuple_ = (self.filename,
                       self.program_id,
                       self.ac_id,
                       self.pipeline_element,
                       self.datetime,
                       self.number
                      )

In [35]:
def add_associations_to_db(assn_list, c, conn):
    
    columns = ['file',
               'program_id',
               'ac_id',
               'pipeline',
               'datetime',
               'number'
              ]
    
    df = pd.DataFrame()
    for a in sorted(assn_list):
        assn = JWSTAssociation(a)
        if assn.formatted:
            assn_fields = list(assn.tuple_)
            print(assn_fields)
            new_row = pd.DataFrame(columns=columns, data=[assn_fields])
            df = df.append(new_row, ignore_index=True)
        else:
            continue
                
    df.to_sql("associations", conn, if_exists="replace", dtype='string')
    return df

In [36]:
print(ASSN)
frame2 = add_associations_to_db(ASSN, c, conn)

['._test_asn17.json', 'extract1d_pars.json', 'jw00035_o001_20170706t153400_spec2_001_asn.json', 'jw10001_20160520T114200_dither_001_asn.json', 'jw111110-a3001_20171109T145456_spec2_001_asn.json', 'jw111110-a3001_20171109T145456_spec2_001_asn.json', 'jw80600-a3002_20170227t160430_spec2_001_asn.json', 'jw80600-a3002_20170227t160430_spec3_001_asn.json', 'jw80600-a3002_20170227t160430_spec3_001_asn.json', 'jw87600-a3001_20170327t343434_ami3_001_asn.json', 'jw87600-a3001_20170327t343434_ami3_001_asn.json', 'jw87600-a3001_20170527t111213_tso3_001_asn.json', 'jw87600-a3001_20171109T145456_spec2_001_asn.json', 'jw87600-a3001_20171109T145456_spec2_001_asn.json', 'jw93065-a3001_20170511t111213_tso3_001_asn.json', 'jw94015_20160524T082730_mosaic_001_asn.json', 'jw95065-a3001_20170309t141001_spec2_001_asn.json', 'jw95065-a3001_20170309t141001_spec3_001_asn.json', 'jw95065-a3001_20170309t141001_spec3_001_asn.json', 'jw96090-a3001_20160415t035424_image3_001_asn.json', 'jw96090-a3001_20160415t035424_

In [37]:
frame2

,file,program_id,ac_id,pipeline,datetime,number
0,jw111110-a3001_20171109T145456_spec2_001_asn.json,111110,a3001,spec2,2017 Nov 09,001
1,jw111110-a3001_20171109T145456_spec2_001_asn.json,111110,a3001,spec2,2017 Nov 09,001
2,jw80600-a3002_20170227t160430_spec2_001_asn.json,80600,a3002,spec2,2017 Feb 27,001
3,jw80600-a3002_20170227t160430_spec3_001_asn.json,80600,a3002,spec3,2017 Feb 27,001
4,jw80600-a3002_20170227t160430_spec3_001_asn.json,80600,a3002,spec3,2017 Feb 27,001
5,jw87600-a3001_20170327t343434_ami3_001_asn.json,87600,a3001,ami3,2017 Mar 27,001
6,jw87600-a3001_20170327t343434_ami3_001_asn.json,87600,a3001,ami3,2017 Mar 27,001
7,jw87600-a3001_20170527t111213_tso3_001_asn.json,87600,a3001,tso3,2017 May 27,001
8,jw87600-a3001_20171109T145456_spec2_001_asn.json,87600,a3001,spec2,2017 Nov 09,001
9,jw87600-a3001_20171109T145456_spec2_001_asn.json,87600,a3001,spec2,2017 Nov 09,001


In [102]:
row = c.execute('SELECT program_id FROM associations WHERE program_id=?', ['96090'])
for p in row:
    print(p)
print(row.rowcount)

(96090,)
(96090,)
-1


In [130]:
def get_exposure_programs_list(c):
    programs = []
    for row in c.execute('SELECT program_id FROM products'):
        programs.append(str(row[0]))
    programs = list(set(programs))
    return programs

In [142]:
PRODUCT_COLUMNS = ['file',
                   'assn_number',
                   'program_id',
                   'instrument',
                   'detector',
                   'optical_elements',
                   'target_id',
                   'source_id',
                   'obs_number',
                   'visit_number',
                   'visit_group',
                   'parallel_seq',
                   'activity_number',
                   'exposure_number',
                   'suffix',
                   'description',
                   'units',
                   'stage'
                  ]

ASSN_COLUMNS = ['file',
                'program_id',
                'ac_id',
                'pipeline',
                'datetime',
                'number'
               ]

In [158]:
def pair_associations_and_exposures(programs, c, conn):
    results = {}
    for p in programs:
        association_query = c.execute('SELECT * FROM associations WHERE program_id=?', [p])
        association_query = c.fetchall()
        associations = [aq[1:] for aq in association_query]
        if len(associations) > 0:
            exposure_query = c.execute('SELECT * FROM products WHERE program_id=?', [p])
            exposure_query = c.fetchall()
            exposures = [eq[1:] for eq in exposure_query]
            print(associations)
            associations_frame = pd.DataFrame(associations, columns=ASSN_COLUMNS)
            exposures_frame = pd.DataFrame(exposures, columns=PRODUCT_COLUMNS)
            associations_title = "{0}_associations".format(p)
            exposures_title = "{0}_exposures".format(p)
            results[p] = {'associations': associations_frame, 'exposures': exposures_frame}
            associations_frame.to_sql(associations_title, conn, if_exists="replace", dtype='string')
            exposures_frame.to_sql(exposures_title, conn, if_exists="replace", dtype='string')
        else:
            print("No association found for {0}".format(p))
    return results

In [159]:
def write_db_to_disk(new_db, c, conn):
    try:
        os.remove(new_db)
    except OSError:
        pass
    
    new_conn = sqlite3.connect(new_db)
    
    query = "".join(line for line in conn.iterdump())

    new_conn.executescript(query)
    
    new_conn.close()

In [160]:
programs = get_exposure_programs_list(c)
results = pair_associations_and_exposures(programs, c, conn)
write_db_to_disk('mydir_files.db', c, conn)

No association found for 11111
[('jw87600-a3001_20170327t343434_ami3_001_asn.json', 87600, 'a3001', 'ami3', '2017 Mar 27', 1), ('jw87600-a3001_20170327t343434_ami3_001_asn.json', 87600, 'a3001', 'ami3', '2017 Mar 27', 1), ('jw87600-a3001_20170527t111213_tso3_001_asn.json', 87600, 'a3001', 'tso3', '2017 May 27', 1), ('jw87600-a3001_20171109T145456_spec2_001_asn.json', 87600, 'a3001', 'spec2', '2017 Nov 09', 1), ('jw87600-a3001_20171109T145456_spec2_001_asn.json', 87600, 'a3001', 'spec2', '2017 Nov 09', 1)]
[('jw80600-a3002_20170227t160430_spec2_001_asn.json', 80600, 'a3002', 'spec2', '2017 Feb 27', 1), ('jw80600-a3002_20170227t160430_spec3_001_asn.json', 80600, 'a3002', 'spec3', '2017 Feb 27', 1), ('jw80600-a3002_20170227t160430_spec3_001_asn.json', 80600, 'a3002', 'spec3', '2017 Feb 27', 1)]
No association found for test1
No association found for 84600
No association found for 94015
No association found for 11001
No association found for al
[('jw93065-a3001_20170511t111213_tso3_001_asn

In [ ]:
c.close()
conn.close()

In [ ]:
import time

x = '20160415T145456'
y = time.strptime(x, '%Y%m%dT%H%M%S')
print(y)
z = time.strftime("%Y %b %d, %X", y)
print(z)